# Leaflet y folium

Leaflet es una librería de código abierto escrita en javascript diseñada con el objetivo de producir mapas interactivos compatibles con dispositivos móviles. Folium es un wrapper de Leaflet para Python.

In [0]:
# Instalamos folium
!pip install folium

In [1]:
# Importamos las principales librerias que usuaremos
import numpy as np
import pandas as pd
import folium
import branca.colormap as cm
from folium.plugins import FastMarkerCluster
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
import branca.colormap as cm
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# Verificamos la versión de folium a utilizar
folium.__version__

## Cargamos los datos

**IMPORTANTE:** Cargar el archivo `"Movilidad_bogota_2015.csv"` en el sistema de archivos de Colab.

Estos datos corresponden a la caracterización de la movilidad en Bogotá en el año 2015, realizada por la secretaría de movilidad y tomada de la página de datos abiertos Colombia disponible en el siguiente enlace: https://www.datos.gov.co/Transporte/Encuesta-de-movilidad-de-Bogot-2015-Caracterizaci-/mvbb-bn7j. Trabajaremos los ejemplos siguientes con estos datos.


In [0]:
# Cargamos los datos con los que trabajaremos en un DataFrame de Pandas
data = pd.read_csv("Movilidad_bogota_2015.csv")

## Exploración inicial de datos

In [0]:
# Información acerca del data frame
data.info()

In [0]:
# Observamos las primeras filas 
data.head(5)

## Limpieza de datos

In [0]:
# Las coordenadas geográficas de Bogotá 
Bog_lat_long = (4.624335, -74.063644)
Bog_lat_long

In [0]:
# Observamos algunos de los datos de latitud y longitud del dataset
(data.iloc[0, 15], data.iloc[0, 17])

Como vemos los datos en el dataset no se encuentran correctamente formateados, así que los formateamos correctamente.

In [0]:
data["LATITUD_ORIGEN"]=data["LATITUD_ORIGEN"]*(10**-13)
data["LATITUD_DESTINO"]=data["LATITUD_DESTINO"]*(10**-13)
data["LONGITUD_ORIGEN"]=data["LONGITUD_ORIGEN"]*(10**-12)
data["LONGITUD_DESTINO"]=data["LONGITUD_DESTINO"]*(10**-12)

In [0]:
# Verificamos ahora el formato de los datos
(data.iloc[0, 15], data.iloc[0, 17])

In [0]:
# Eliminamos los valores nan
data.dropna(inplace=True, how="any", subset=["LATITUD_ORIGEN", "LATITUD_DESTINO", "LONGITUD_ORIGEN",
                                            "LONGITUD_DESTINO"], axis=0)

In [0]:
# Contamos el numero de filas sin NaN values 
data["LATITUD_DESTINO"].count()

## Cración del mapa

In [0]:
# Creamos nuestro primer mapa con folium con los datos de la primera encuesta
test_map = folium.Map(location=Bog_lat_long)

In [0]:
# Visualizamos el mapa de Bogotá
test_map

In [0]:
# Los marcadores son de dos tipos principales marcadores simples como el siguiente
folium.Marker(location=(data["LATITUD_ORIGEN"][0], data["LONGITUD_ORIGEN"][0]), 
              popup="Marcador simple Origen", tooltip="Click me!").add_to(test_map)
# También podremos tener marcadores 
folium.vector_layers.CircleMarker(location=(data["LATITUD_DESTINO"][0], data["LONGITUD_DESTINO"][0]), 
                                  fill=True, fill_color="blue", radius=10, popup="Marcador circular destino", tooltip="Click me").add_to(test_map)
test_map

### Mapa de orígenes

In [0]:
# Visualizamos los origenes destino en el mapa para ello creamos la siguiente función
def setPoint(point, icon_color, **kwargs):
  popup_text = ""
  # Procesamos la información addicional
  for k,v in kwargs.items():
    popup_text += "{}: {}\n".format(k,v)
  # Definimos un marcador
  folium.Marker(location=point, popup=popup_text, icon=folium.Icon(color=icon_color, icon='ok-sign')).add_to(test_map)

In [0]:
# Dibujar los puntos en el mapa con datos con información en el popup para visualizar mejor lo limitamos a 50 ejemplos
for i in list(data.index)[:50]:
  setPoint(point=(data.loc[i, "LATITUD_ORIGEN"], data.loc[i, "LONGITUD_ORIGEN"]), icon_color="green", 
           motivo_viaje=data.loc[i, "MOTIVOVIAJE"], hora_inicio=data.loc[i,"HORA_INICIO"], 
           hora_fin=data.loc[i, "HORA_FIN"], medio=data.loc[i, "MEDIO_PREDOMINANTE"])

In [0]:
# Guardamos el mapa
test_map.save("Origen_map.html")

In [0]:
# Visualizamos el mapa
test_map

### Mapa de destinos

In [0]:
# Volvemos a crear el mapa esta vez com más zoom
test_map = folium.Map(location=Bog_lat_long, zoom_start=13)

# Dibujar los puntos destino en el mapa con datos con información en el popup para visualizar mejor lo limitamos a 50 ejemplos
for i in list(data.index)[:50]:
  setPoint(point=(data.loc[i, "LATITUD_DESTINO"], data.loc[i, "LONGITUD_DESTINO"]), icon_color="red", 
           motivo_viaje=data.loc[i, "MOTIVOVIAJE"], hora_inicio=data.loc[i,"HORA_INICIO"], 
           hora_fin=data.loc[i, "HORA_FIN"], medio=data.loc[i, "MEDIO_PREDOMINANTE"])

In [0]:
test_map

## ClusterMap

Como pudimos observar en el gráfico superior hay bastantes puntos de origen y destino así que visualizarlos resulta complicado y poco claro, por ello los agruparemos de acuerdo a sus coordenadas. 

In [0]:
mapa_2 = folium.Map(location=Bog_lat_long, zoom_start=12, control_scale=True,prefer_canvas=True)
FastMarkerCluster(name ="Cluster de coordenadas" ,data=list(zip(data.LATITUD_ORIGEN,data.LONGITUD_ORIGEN))).add_to(mapa_2)
mapa_2.save("mapa_con_cluster_origen.html")


In [0]:
mapa_2

In [0]:
# Ahora realizaremos un agrupamiento para 
mapa_3 = folium.Map(location=Bog_lat_long, zoom_start=10, control_scale=True,prefer_canvas=True)
FastMarkerCluster(name ="Cluster de coordenadas" ,data=list(zip(data.LATITUD_DESTINO,data.LONGITUD_DESTINO))).add_to(mapa_3)
mapa_3.save("mapa_con_cluster_destino.html")
mapa_3

## Heatmap

Heatmap para visualizar los orígenes en las horas pico de la mañana.

In [0]:
inicio = pd.to_datetime("04:00")
fin = pd.to_datetime("09:00")

In [0]:
# Convertimos las horas de inicio y fin de string a time values
data["HORA_INICIO"] = pd.to_datetime(data["HORA_INICIO"], errors='ignore')
data["HORA_FIN"] = pd.to_datetime(data["HORA_FIN"], errors='ignore')

In [0]:
hora_pico_am=data[(data["HORA_INICIO"]>inicio) & (data["HORA_INICIO"]<fin)]

In [0]:
hora_pico_am.info()

In [0]:
# Mapa de calor 
map_bog = folium.Map(location=Bog_lat_long,
                    zoom_start = 13) 

In [0]:
# Heat data 
heat_data = [[row["LATITUD_ORIGEN"], row["LONGITUD_ORIGEN"]] for index, row in hora_pico_am.iterrows()]

In [0]:
# Añadimos los datos al mapa
HeatMap(heat_data).add_to(map_bog)
map_bog.save("mapa_con_heat_map.html")
#Visualizamos el mapa
map_bog

## Ideas adicionales 



Podemos realizar varias visualizaciones adicionales muy útiles como:
* Aplicando un filtro en el dataset seleccionar aquellos viajes realizados en transporte público y visualizarlos.
* Utilizando los datos de los polígonos que conforman las localidades en https://bogota-laburbano.opendatasoft.com/explore/dataset/poligonos-localidades/export/?flg=es&location=8,4.2841,-74.21816&basemap=jawg.streets podemos agrupar los viajes de acuerdo a la localidad de origen y la localidad de destino y así visualizar los viajes interlocalidades.
* Filtrar los datos de acuerdo a la hora pico de la tarde o la mañana y agruparlos para determinar cuales son los orígenes destinos de estas horas. 
* Utilizando los datos geográficos de las localidades realizar la visualización de un mapa cloroplético de acuerdo al número de viajes, utilizando el siguiente ejemplo como guía https://python-graph-gallery.com/292-choropleth-map-with-folium/

## Referencias 


* https://deparkes.co.uk/2016/06/24/folium-marker-clusters/
* https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-data
* https://python-visualization.github.io/folium/quickstart.html#Getting-Started
* https://mappinggis.com/2018/10/folium-utilizando-leaflet-con-python/
* https://leafletjs.com/examples/geojson/
* https://deparkes.co.uk/2016/06/03/plot-lines-in-folium/
* http://bl.ocks.org/wrobstory/5609747